In [1]:
%matplotlib ipympl
# Basic libraries for data manipulation
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import multiprocessing as mp

# Neuron libraries
from neuron import h
from neuron import load_mechanisms
import neuron

import ipywidgets as wdg
from tqdm import tqdm, tqdm_notebook

#Load main NEURON modules
load_mechanisms('/work/nrn-7.4/x86_64/bin/')
h.load_file('stdrun.hoc')
h('objref nil')

class BallAndStick(object):
    
    def __init__(self, E_PAS = -75.0, Rm = 10000.0, Cm = 1.0, Ra = 150.0, celsius = 23, dend_nseg = 11, 
                 soma_diam = 25, dend_length = 25, dend_diam = 1):
         # Initialise ephys parameters
        self.E_PAS = E_PAS
        self.Rm = Rm
        self.Cm = Cm
        self.Ra = Ra
        self.CELSIUS = celsius
        
        # Create soma and dendrite and connect them together
        self.soma = h.Section(name="soma")
        self.dend = h.Section(name="dend")
        self.dend.connect(self.soma(1))
        
        # Initialise soma and dendrite diameters
        self.dend.L = dend_length # This makes soma 500 microns squared
        self.soma.diam = soma_diam
        self.dend.diam = dend_diam
        self.soma.L = soma_diam
        
        # Set dendritic segments
        self.dend.nseg = dend_nseg
        
        # Insert conductances
        self.soma = self.add_conductances(self.soma)
        self.dend = self.add_conductances(self.dend)
        
        # Insert tapering of dendrite
#         self.dend = self.taper_diam(self.dend,2,1)
    
    def taper_diam(self, sec,zero_bound,one_bound):
        for (num_sec, d) in zip(sec, np.linspace(zero_bound, one_bound, sec.nseg)):
            num_sec.diam=d
            
        return sec

    def add_conductances(self, nrn_sec):
        nrn_sec.insert('pas')
        nrn_sec.Ra = self.Ra
        nrn_sec.e_pas = self.E_PAS
        nrn_sec.g_pas = 1.0/self.Rm
        for seg in nrn_sec:
            seg.cm = self.Cm
      
        return nrn_sec
    
    def add_AMPA(self, func = h.Exp2Syn, section = h.Section(), locs = [0.5], gmax = 0.5, tau1 = 0.1, tau2 = 1 ):
        self.AMPA_syns, self.AMPA_ncs = [], []
        for syn_no in range(len(locs)):
            SYN = func(float(locs[syn_no]), sec = section)
            SYN.tau1 = tau1
            SYN.tau2 = tau2
            if type(gmax) == list or type(gmax) == np.ndarray:
                NC = h.NetCon(h.nil, SYN, 0, 0, gmax[syn_no])
            else:
                NC = h.NetCon(h.nil, SYN, 0, 0, gmax)
                
            self.AMPA_syns.append(SYN), self.AMPA_ncs.append(NC)
            
    def add_NMDA(self,locs=[0.5],gmax=[1],rel = [20]):
        self.NMDAlist = []
        self.preNMDA_list = []
        for loc in enumerate(locs):
            PRE = h.Section()
            PRE.diam = 1.0 ; PRE.L=1.0
            PRE.insert('rel')
            PRE.dur_rel = 0.5
            PRE.amp_rel = 2.0
            PRE.del_rel = rel[loc[0]]
            NMDA = h.NMDA_Mg_T(self.dend(loc[1]))
            NMDA.gmax = gmax[loc[0]]
            h.setpointer(PRE(0.5).rel._ref_T,'C',NMDA)
            self.preNMDA_list.append(PRE)
            self.NMDAlist.append(NMDA)
            
    def simulate(self, v_init = -75, t_stop=200, NMDA=False):
        """ Run the main simulation. Accepts AMPA only or with NMDA depending on passing of the NMDA parameters.
        Records from soma."""
        
        self.vec = {}
        for type_rec in "vrec","trec":
            self.vec[type_rec] = h.Vector()

        self.vec["trec"].record(h._ref_t)
        self.vec["vrec"].record(self.soma(0.5)._ref_v)
        
        if NMDA==True:
            self.NMDAgrec, self.NMDAirec = [], []
            for chan in np.arange(0, len(self.NMDAlist)):
                loc = self.NMDAlist[chan].get_loc()
                h.pop_section()
                self.NMDAgrec.append(h.Vector())
                self.NMDAgrec[chan].record(self.NMDAlist[chan]._ref_g)
                self.NMDAirec.append(h.Vector())
                self.NMDAirec[chan].record(self.NMDAlist[chan]._ref_i)
        
        h.celsius = self.CELSIUS
        h.finitialize(v_init)
        neuron.run(t_stop)
        
    def run_IN_OUT(self, base_AMPA = 0.0005, base_NMDA = 8000, synapses = 16, base_step = 2, 
                   nmda_gradient_top = 1, nmda_gradient_bot = 1, ampa_gradient_top = 1, ampa_gradient_bot = 1,
                   IN_scale_ampa = 1, IN_scale_nmda = 1, OUT_scale_ampa = 1, OUT_scale_nmda = 1,
                   syn_placement_bot = 0, syn_placement_top = 1, dendrite = 13, base_time = 20):
        
        # First reconstruct parameters
        syn_placement = np.linspace(syn_placement_bot,syn_placement_top,synapses)
                    
        for seq_type in ["IN","OUT"]:

            # Define synapse placement for IN and OUT sequences
            if seq_type == "IN":
                syn_sequence = np.linspace(base_time,base_time+synapses*base_step,synapses)[::-1]
            if seq_type == "OUT":
                syn_sequence = np.linspace(base_time,base_time+synapses*base_step,synapses)

            self.activation_pattern = enumerate(syn_sequence)
            
            ampa_gradient = np.linspace(1,1,synapses)
            nmda_gradient = np.linspace(1,1,synapses)
            
            # Scale synapses 
            ampa_gmax = base_AMPA*ampa_gradient*IN_scale_ampa
            nmda_gmax = base_NMDA*nmda_gradient*IN_scale_nmda

            #Add synapses
            self.add_AMPA(locs=syn_placement,gmax=ampa_gmax,tau1=0.1,tau2=1, section =self.dend)
            self.add_NMDA(locs=syn_placement,gmax=nmda_gmax,rel=syn_sequence)

            # Run the simulation and plot
            fih = h.FInitializeHandler(1,netcon_events)
            self.simulate(t_stop=500,NMDA=True)

            if seq_type == "IN":
                IN = np.array(self.vec["vrec"])

            if seq_type == "OUT":
                OUT = np.array(self.vec["vrec"])
                
        return IN,OUT

In [3]:
class BS_heatmap:
    def __init__(self, file_path, base_step = 8.0, dend_length = 50.0, soma_diam=50.0,dend_diam = 1,split_analysis = False, 
                 min_epsp_threshold=0.2, max_epsp_threshold=0.5):
        self.xdata = []
        self.ydata = []
        self.current_point = [] # Variable to store selected point on a graph
        self.hm_ranges = {}
        self.cbar = []
        
        self.dend_length = dend_length
        self.soma_diam = soma_diam
        
        # Set thresholds
        self.min_epsp_threshold, self.max_epsp_threshold = min_epsp_threshold, max_epsp_threshold
        
        # Initialise data and other figures
        self.df = pd.read_csv(file_path)
        self.fig, (self.ax_heat, self.ax_line) = plt.subplots(nrows=1,ncols=2)
        
        self.heatmap_plot(base_step = base_step,dend_length=dend_length,soma_diam=soma_diam,
                          split_analysis = split_analysis)

        self.cid = self.fig.canvas.mpl_connect("button_press_event",self)

        
    def __call__(self,event):
        # Initialise first point or update if first point has been plotted
        if self.current_point != []:
            self.current_point[0].set_xdata(event.xdata)
            self.current_point[0].set_ydata(event.ydata)
        else:
            self.current_point = self.ax_heat.plot(event.xdata,event.ydata,"mo",ms=5,mec="none")
          
        ampa = ((self.hm_ranges["ampa"][1]-self.hm_ranges["ampa"][0])/50)*event.ydata+self.hm_ranges["ampa"][0]
        nmda = ((self.hm_ranges["nmda"][1]-self.hm_ranges["nmda"][0])/50)*event.xdata+self.hm_ranges["nmda"][0]
        
        self.ax_line.cla()        
        
        # Setup variables
        MODEL.dend.L = self.dend_length
        MODEL.soma.diam = self.soma_diam
        MODEL.soma.L = self.soma_diam
        IN, OUT = MODEL.run_IN_OUT(base_AMPA=ampa, base_NMDA=nmda, base_step=self.base_step)
        print("Soma L:",MODEL.soma.L, "Soma diam:",MODEL.soma.diam,"MODEL.dend.L",MODEL.dend.L,"MODEL.dend.diam:",
             MODEL.dend.diam)
        
        # Show on plot        
        self.ax_line.plot(IN, "r")
        self.ax_line.plot(OUT, "b")
        
    def heatmap_plot(self, base_step = 8.0, dend_length = 50.0, soma_diam = 50.0,split_analysis = False):
        
        self.hm_data = self.df[(self.df["base_step"] == float(base_step)) & (self.df["length"] == float(dend_length)) 
                               & (self.df["soma_diam"] == float(soma_diam))]
        
        #### No dend diam in passed arguments!!!!!!
        
        # Set global variables
        self.base_step, self.length, self.soma_diam = base_step, dend_length, soma_diam
        
        self.cbar = self.ax_heat.imshow(self.hm_data["diff"].values.reshape(50,50))
        self.fig.colorbar(self.cbar)
        
        # Find the current ranges of the plot
        self.hm_ranges["ampa"] = (self.df["ampa"].min(), self.df["ampa"].max())
        self.hm_ranges["nmda"] = (self.df["nmda"].min(), self.df["nmda"].max())
        
        # Add appropriate ranges to the plot
        self.ax_heat.set_xticks([0,50]),self.ax_heat.set_yticks([0,50])
        self.ax_heat.set_xticklabels(self.hm_ranges["nmda"]), self.ax_heat.set_yticklabels(self.hm_ranges["ampa"])
        
        if split_analysis:
            self.parallel_run_accepted()
            masked_data = self.accepted_vector.reshape(50,50)
            masked_data = np.ma.masked_where(masked_data > 0.9, masked_data)
            self.ax_heat.imshow(masked_data, alpha = 0.7)
            
        self.fig.canvas.draw()
        
    def get_max_potentiation(ampa_base = 0.001, nmda_base = 1000, max_potentiation =1):
        selected_potentiations = self.hm_data[(self.hm_data["ampa"] >= ampa_base) 
                                       & (self.hm_data["ampa"] <= ampa_base*(1+max_potentiation))]

        # Check for out
        out_wins = ((selected_potentiations["IN"] - selected_potentiations["OUT"])<0).sum()
        in_wins = ((selected_potentiations["IN"] - selected_potentiations["OUT"])>0).sum()

        return [out_wins >0, in_wins > 0]
        
    def __run_split_analysis__(self,splitted_df,max_potentiation=1):
        accepted_df = []
        for row in tqdm(splitted_df.iterrows()):
            epsp = np.max(MODEL.run_IN_OUT(base_AMPA=row[1]["ampa"], base_NMDA=row[1]["nmda"],synapses=1)) + 75
            if (epsp < self.min_epsp_threshold) | (epsp > self.max_epsp_threshold):
                selected_potentiations = self.hm_data[(self.hm_data["ampa"] >= row[1]["ampa"]) 
                                                       & (self.hm_data["ampa"] <= row[1]["ampa"]*(1+max_potentiation))]

                # Check for out
                out_wins = ((selected_potentiations["IN"] - selected_potentiations["OUT"])<0).sum()
                in_wins = ((selected_potentiations["IN"] - selected_potentiations["OUT"])>0).sum()                
                accepted_df.append([row[1]["ampa"],row[1]["nmda"],epsp,False,
                                    MODEL.dend.L, MODEL.soma.L,MODEL.dend.diam, MODEL.soma.diam, out_wins, in_wins])


            else:
                selected_potentiations = self.hm_data[(self.hm_data["ampa"] >= row[1]["ampa"]) 
                                                       & (self.hm_data["ampa"] <= row[1]["ampa"]*(1+max_potentiation))]

                # Check for out
                out_wins = ((selected_potentiations["IN"] - selected_potentiations["OUT"])<0).sum()
                in_wins = ((selected_potentiations["IN"] - selected_potentiations["OUT"])>0).sum()
                
                accepted_df.append([row[1]["ampa"],row[1]["nmda"],epsp,True,
                                    MODEL.dend.L, MODEL.soma.L,MODEL.dend.diam, MODEL.soma.diam, out_wins, in_wins])
            
        return pd.DataFrame(accepted_df, columns = ["ampa","nmda","epsp","threshold","dend_length",
                                                    "soma_length","dend_diam","soma_diam","out_pot","in_pot"])
    
    def parallel_run_accepted(self):
        """ Create a masked array to overlay for discovered heatmap to show appropriate EPSP values."""
        
        cores = mp.cpu_count()
        pool = mp.Pool(processes=cores)
        self.accepted_df = pd.concat(pool.map(self.__run_split_analysis__,np.array_split(self.hm_data,cores)))
        self.accepted_df["accepted"] = (self.accepted_df["threshold"] == True) & (self.accepted_df["out_pot"] > 0) & (self.accepted_df["in_pot"] > 0)
        self.accepted_vector = self.accepted_df["accepted"]
        
        pool.close()
        
def netcon_events():
    """ Launch AMPA synapses with netcon events. """
    for syn_event in MODEL.activation_pattern:
        MODEL.AMPA_ncs[syn_event[0]].event(float(syn_event[1]))

MODEL = BallAndStick("/shared/simulation_results_v2.csv", dend_length=100, soma_diam=50, dend_diam=1)
bs_hm = BS_heatmap(base_step = 2.0, dend_length = 100, split_analysis=True,min_epsp_threshold=0.2, max_epsp_threshold=0.5)
        

ValueError: bad value

In [14]:
bs_hm.hm_ranges

{'ampa': (0.0001, 0.0015), 'nmda': (1000.0, 16000.0)}

### Incomplete - Work in progress !!!

In [ ]:
class BS_heatmap:
    def __init__(self, file_path, base_step = 8.0, dend_length = 50.0, soma_diam=50.0,dend_diam = 1,split_analysis = False, 
                 min_epsp_threshold=0.2, max_epsp_threshold=0.5):
        self.xdata = []
        self.ydata = []
        self.current_point = [] # Variable to store selected point on a graph
        self.hm_ranges = {}
        self.cbar = []
        
        self.dend_length = dend_length
        self.soma_diam = soma_diam
        
        # Set thresholds
        self.min_epsp_threshold, self.max_epsp_threshold = min_epsp_threshold, max_epsp_threshold
        
        # Initialise data and other figures
        self.df = pd.read_csv(file_path)
        self.fig, (self.ax_heat, self.ax_line) = plt.subplots(nrows=1,ncols=2)
        
        self.heatmap_plot(base_step = base_step,dend_length=dend_length,soma_diam=soma_diam,
                          split_analysis = split_analysis)

        self.cid = self.fig.canvas.mpl_connect("button_press_event",self)

        
    def __call__(self,event):
        # Initialise first point or update if first point has been plotted
        if self.current_point != []:
            self.current_point[0].set_xdata(event.xdata)
            self.current_point[0].set_ydata(event.ydata)
        else:
            self.current_point = self.ax_heat.plot(event.xdata,event.ydata,"mo",ms=5,mec="none")
          
        ampa = ((self.hm_ranges["ampa"][1]-self.hm_ranges["ampa"][0])/50)*event.ydata+self.hm_ranges["ampa"][0]
        nmda = ((self.hm_ranges["nmda"][1]-self.hm_ranges["nmda"][0])/50)*event.xdata+self.hm_ranges["nmda"][0]
        
        self.ax_line.cla()        
        
        # Setup variables
        MODEL.dend.L = self.dend_length
        MODEL.soma.diam = self.soma_diam
        MODEL.soma.L = self.soma_diam
        IN, OUT = MODEL.run_IN_OUT(base_AMPA=ampa, base_NMDA=nmda, base_step=self.base_step)
        print("Soma L:",MODEL.soma.L, "Soma diam:",MODEL.soma.diam,"MODEL.dend.L",MODEL.dend.L,"MODEL.dend.diam:",
             MODEL.dend.diam)
        
        # Show on plot        
        self.ax_line.plot(IN, "r")
        self.ax_line.plot(OUT, "b")
        
    def heatmap_plot(self, base_step = 8.0, dend_length = 50.0, soma_diam = 50.0,split_analysis = False):
        
        self.hm_data = self.df[(self.df["base_step"] == float(base_step)) & (self.df["length"] == float(dend_length)) 
                               & (self.df["soma_diam"] == float(soma_diam))]
        
        #### No dend diam in passed arguments!!!!!!
        
        # Set global variables
        self.base_step, self.length, self.soma_diam = base_step, dend_length, soma_diam
        
        self.cbar = self.ax_heat.imshow(self.hm_data["diff"].values.reshape(50,50))
        self.fig.colorbar(self.cbar)
        
        # Find the current ranges of the plot
        self.hm_ranges["ampa"] = (self.df["ampa"].min(), self.df["ampa"].max())
        self.hm_ranges["nmda"] = (self.df["nmda"].min(), self.df["nmda"].max())
        
        # Add appropriate ranges to the plot
        self.ax_heat.set_xticks([0,50]),self.ax_heat.set_yticks([0,50])
        self.ax_heat.set_xticklabels(self.hm_ranges["nmda"]), self.ax_heat.set_yticklabels(self.hm_ranges["ampa"])
        
        if split_analysis:
            self.parallel_run_accepted()
            masked_data = self.accepted_vector.reshape(50,50)
            masked_data = np.ma.masked_where(masked_data > 0.9, masked_data)
            self.ax_heat.imshow(masked_data, alpha = 0.7)
            
        self.fig.canvas.draw() 
    
    def __run_split_analysis__(self,splitted_df,potentiation_params={"min":2.0, "max":25.0}):
        accepted_df = []
        for row in tqdm(splitted_df.iterrows()):
            
            # Get simulations
            epsp = np.max(MODEL.run_IN_OUT(base_AMPA=row[1]["ampa"], base_NMDA=row[1]["nmda"],synapses=1)) + 75
            
            # Test if epsp mini is within accepted min and max parameters
            test_epsp = not (epsp < self.min_epsp_threshold) | (epsp > self.max_epsp_threshold)
            
            # Add all the parameters to the data frame
            accepted_df.append([row[1]["ampa"],row[1]["nmda"],epsp,test_epsp,
                                MODEL.dend.L, MODEL.soma.L,MODEL.dend.diam, MODEL.soma.diam,
                                row[1]["IN"],row[1]["OUT"]])
            
        df = pd.DataFrame(accepted_df, columns = ["ampa","nmda","epsp","mini_threshold",
                                                    "dend_length","soma_length","dend_diam","soma_diam","IN","OUT"])
        
        # Create accepted vector for all tested conditions
        df["threshold"] = df["mini_threshold"] 
        
        return df
    
    def parallel_run_accepted(self):
        """ Create a masked array to overlay for discovered heatmap to show appropriate EPSP values."""
        
        cores = mp.cpu_count()
        pool = mp.Pool(processes=cores)
        self.accepted_df = pd.concat(pool.map(self.__run_split_analysis__,np.array_split(self.hm_data,cores)))
        self.accepted_vector = self.accepted_df["threshold"]
        
        pool.close()
        
def netcon_events():
    """ Launch AMPA synapses with netcon events. """
    for syn_event in MODEL.activation_pattern:
        MODEL.AMPA_ncs[syn_event[0]].event(float(syn_event[1]))

MODEL = BallAndStick(dend_length=100, soma_diam=50, dend_diam=1)
bs_hm = BS_heatmap("/shared/simulation_results_v2.csv", base_step = 2.0, dend_length = 100, split_analysis=True)
        

/usr/local/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

1918it [02:45, 11.56it/s]

In [10]:
bs_hm.accepted_df.head()

,ampa,nmda,epsp,mini_threshold,dend_length,soma_length,dend_diam,soma_diam,IN,OUT,threshold
0,0.0001,1000.000000,0.095188,False,100.0,50.0,1.0,50.0,-74.040785,-74.057679,False
1,0.0001,1306.122449,0.096408,False,100.0,50.0,1.0,50.0,-73.903606,-73.920856,False
2,0.0001,1612.244898,0.097717,False,100.0,50.0,1.0,50.0,-73.761642,-73.779969,False
3,0.0001,1918.367347,0.099141,False,100.0,50.0,1.0,50.0,-73.614373,-73.634489,False
4,0.0001,2224.489796,0.100724,False,100.0,50.0,1.0,50.0,-73.461114,-73.483627,False


In [27]:
scaling_params = {"min":1,"max":2.5}

xf = bs_hm.accepted_df
xf["diff"] = xf["IN"]-xf["OUT"]

xf["PotMin"], xf["PotMax"] = xf["IN"]*scaling_params["min"], xf["IN"]*scaling_params["max"] 

In [41]:
xf[xf.threshold]["OUTpot_test"].sum()/xf[xf.threshold].shape[0]

0.72368421052631582

In [42]:
xf[xf.threshold]["INpot_test"].sum()/xf[xf.threshold].shape[0]

1.0

In [37]:
# Test OUT
xf["OUTpot_test"] = xf.apply(lambda x: any(xf[(xf["IN"]>x["PotMin"]) & (xf["IN"]<x["PotMax"])]["diff"] < 0), axis=1)
xf["INpot_test"] = xf.apply(lambda x: any(xf[(xf["IN"]>x["PotMin"]) & (x["IN"]<x["PotMax"])]["diff"] > 0), axis=1)



,ampa,nmda,epsp,mini_threshold,dend_length,soma_length,dend_diam,soma_diam,IN,OUT,threshold,diff,PotMin,PotMax,OUTpot_test,INpot_test
0,0.0001,1000.000000,0.095188,False,100.0,50.0,1.0,50.0,0.959215,0.942321,False,0.016894,0.959215,2.398037,False,True
1,0.0001,1306.122449,0.096408,False,100.0,50.0,1.0,50.0,1.096394,1.079144,False,0.017250,1.096394,2.740985,False,True
2,0.0001,1612.244898,0.097717,False,100.0,50.0,1.0,50.0,1.238358,1.220031,False,0.018327,1.238358,3.095896,False,True
3,0.0001,1918.367347,0.099141,False,100.0,50.0,1.0,50.0,1.385627,1.365511,False,0.020116,1.385627,3.464066,True,True
4,0.0001,2224.489796,0.100724,False,100.0,50.0,1.0,50.0,1.538886,1.516373,False,0.022513,1.538886,3.847215,True,True
5,0.0001,2530.612245,0.107117,False,100.0,50.0,1.0,50.0,1.699127,1.673975,False,0.025152,1.699127,4.247817,True,True
6,0.0001,2836.734694,0.116943,False,100.0,50.0,1.0,50.0,1.867958,1.841341,False,0.026617,1.867958,4.669895,True,True
7,0.0001,3142.857143,0.127055,False,100.0,50.0,1.0,50.0,2.048136,2.023756,False,0.024380,2.048136,5.120340,True,True
8,0.0001,3448.979592,0.137338,False,100.0,50.0,1.0,50.0,2.241769,2.220963,False,0.020806,2.241769,5.604422,True,True
9,0.0001,3755.102041,0.147737,False,100.0,50.0,1.0,50.0,2.448018,2.430925,False,0.017093,2.448018,6.120045,True,True


## Simple example of multiprocessing

In [16]:
import multiprocessing as mp

def __run_split_analysis__(self,splitted_df):
    accepted_vector = []
    for row in tqdm(splitted_df.iterrows()):
        epsp = np.max(bs.run_IN_OUT(base_AMPA=row[1]["ampa"], base_NMDA=row[1]["nmda"], synapses = 1)) + 75
        if (epsp < min_epsp_threshold) | (epsp > max_epsp_threshold):
            accepted_vector.append(False)
        else:
            accepted_vector.append(True)

    return accepted_vector


min_epsp_threshold, max_epsp_threshold = 0.2, 0.5

cores = mp.cpu_count()
pool = mp.Pool(processes=cores)

accepted_vector = pool.map(run_split_analysis,np.array_split(bs_hm.hm_data,cores))
accepted_vector = [item for sublist in accepted_vector for item in sublist]

pool.close()


NameError: name 'run_split_analysis' is not defined

In [15]:
bs_hm.hm_data

,Unnamed: 0,ampa,nmda,IN,OUT,diff,base_step,length,soma_diam
78,78,0.0001,1000.000000,-74.040785,-74.057679,0.016894,2.0,100.0,50.0
318,318,0.0001,1306.122449,-73.903606,-73.920856,0.017250,2.0,100.0,50.0
558,558,0.0001,1612.244898,-73.761642,-73.779969,0.018327,2.0,100.0,50.0
798,798,0.0001,1918.367347,-73.614373,-73.634489,0.020116,2.0,100.0,50.0
1038,1038,0.0001,2224.489796,-73.461114,-73.483627,0.022513,2.0,100.0,50.0
1278,1278,0.0001,2530.612245,-73.300873,-73.326025,0.025152,2.0,100.0,50.0
1518,1518,0.0001,2836.734694,-73.132042,-73.158659,0.026617,2.0,100.0,50.0
1758,1758,0.0001,3142.857143,-72.951864,-72.976244,0.024380,2.0,100.0,50.0
1998,1998,0.0001,3448.979592,-72.758231,-72.779037,0.020806,2.0,100.0,50.0
2238,2238,0.0001,3755.102041,-72.551982,-72.569075,0.017093,2.0,100.0,50.0


In [13]:
xf = bs_hm.accepted_df

xf[(xf.nmda == xf.nmda.iloc[17]) & (xf.threshold)]

,ampa,nmda,epsp,threshold,dend_length,soma_length,dend_diam,soma_diam,out_pot,in_pot,accepted
17,0.000100,6204.081633,0.232865,True,100.0,50.0,1.0,50.0,60,140,True
67,0.000129,6204.081633,0.238061,True,100.0,50.0,1.0,50.0,65,185,True
117,0.000157,6204.081633,0.243659,True,100.0,50.0,1.0,50.0,67,233,True
167,0.000186,6204.081633,0.249740,True,100.0,50.0,1.0,50.0,69,281,True
217,0.000214,6204.081633,0.256414,True,100.0,50.0,1.0,50.0,69,331,True
267,0.000243,6204.081633,0.263864,True,100.0,50.0,1.0,50.0,70,380,True
317,0.000271,6204.081633,0.274368,True,100.0,50.0,1.0,50.0,70,430,True
9,0.000300,6204.081633,0.299639,True,100.0,50.0,1.0,50.0,70,480,True
59,0.000329,6204.081633,0.325152,True,100.0,50.0,1.0,50.0,70,530,True
109,0.000357,6204.081633,0.350804,True,100.0,50.0,1.0,50.0,69,581,True


In [ ]:
from skimage.transform import resize
from skimage.io import imread

fig, ax = plt.subplots()
img=imread('/vagrant/parrot.jpg')
imgplot = ax.imshow(resize(img, (50,50)))

masked_data = bs_hm.accepted_vector.reshape(50,50)
masked_data = np.ma.masked_where(masked_data < 0.9, masked_data)
ax.imshow(masked_data, alpha = 0.7)
